# SNOTEL_query

### Script to query SNOTEL sites over study area/period through the ulmo CUAHSI interface and plot SWE/snow depth

Adapted from the [2021 SnowEx Hackweek notebook](https://github.com/snowex-hackweek/website/blob/main/book/tutorials/geospatial/SNOTEL_query.ipynb). 

In [1]:
# use this cell to download any packages not yet installed
# !pip install contextily
# !conda install -c conda-forge ulmo # <- this one may need to be run in the terminal

In [9]:
import os
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import contextily as ctx
import ulmo

In [8]:
# -----latest CUAHSI API endpoint
wsdlurl = 'https://hydroportal.cuahsi.org/Snotel/cuahsi_1_1.asmx?WSDL'

# -----fetch available site metadata from server
sites = ulmo.cuahsi.wof.get_sites(wsdlurl)

# -----store dictionary as Pandas DataFrame
sites_df = pd.DataFrame.from_dict(sites, orient='index').dropna()
sites_df.head()

# ---clean up DataFrame
# convert 'location' column (containing 'longitude' and 'latitude') to Shapely Point objects
sites_df['geometry'] = [Point(float(loc['longitude']), float(loc['latitude'])) for loc in sites_df['location']]
# drop 'location' column (no longer needed)
sites_df = sites_df.drop(columns='location')
# set elevation column datatype to float
sites_df = sites_df.astype({"elevation_m":float})
# sort DataFrame by name
sites_df = sites_df.sort_values(by='name')

# -----view all sites (very big)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
#     print(sites_df)
    
# -----filter to study sites
# Grand Mesa (GM) - two SNOTEL sites
GM1 = sites_df.loc['SNOTEL:622_CO_SNTL']
GM2 = sites_df.loc['SNOTEL:682_CO_SNTL']
# Mores Creek Summit (MCS)
MCS = sites_df.loc['SNOTEL:637_ID_SNTL']

### AOI #1: Grand Mesa, CO

In [ ]:
# bounding box Shapely Polygon
latmin = 38.884619201291905
latmax = 39.11727568585598
lonmin = -107.9
lonmax = -108.24005126953124
AOI1_bb = Polygon([(lonmin, latmin), (lonmax, latmin), (lonmax, latmax), (lonmin, latmax), (lonmin, latmin)])

# convert df to geopandas df
GM1_gdf = gpd.GeoDataFrame(GM1, crs='EPSG:4326')
GM2_gdf = gpd.GeoDataFrame(GM2, crs='EPSG:4326')
